Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 7.71 ms, sys: 662 µs, total: 8.37 ms
Wall time: 11.5 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.1.0.26:8787/status,
Dashboard: http://10.1.0.26:8787/status,Workers: 1
Total threads: 6,Total memory: 6.78 GiB
Status: running,Using processes: False
Comm: inproc://10.1.0.26/8201/1,Workers: 1
Dashboard: http://10.1.0.26:8787/status,Total threads: 6
Started: Just now,Total memory: 6.78 GiB
Comm: inproc://10.1.0.26/8201/4,Total threads: 6
Dashboard: http://10.1.0.26:34607/status,Memory: 6.78 GiB
Nanny: None,


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 956 ms, sys: 81.3 ms, total: 1.04 s
Wall time: 2.39 s


[1.7087072633889095,
 1.6046312123674524,
 1.9705384118171088,
 1.0314476780174844,
 1.5674259417100307,
 1.7425046593241937,
 1.6899621041672803,
 1.1268154453274348,
 1.3181776337313642,
 1.245850774697037,
 1.051322620457947,
 1.9662637572412403,
 1.7785368993965087,
 1.3676486807553632,
 1.8908100311737073,
 1.6452338147529038,
 1.8897823798938194,
 1.0696125009167567,
 1.9228746359408617,
 1.2136341796708716,
 1.568836174644579,
 1.5697804257812917,
 1.3343061717591078,
 1.60033310481872,
 1.4550037966720486,
 1.3466215437851745,
 1.9947939891530369,
 1.89070435557637,
 1.764274145169794,
 1.5412226384940229,
 1.617016976439422,
 1.7358891637665042,
 1.1725872946804778,
 1.5345024078360174,
 1.8253447490481314,
 1.3432629660566333,
 1.0231841232315575,
 1.5389647542712241,
 1.4514538269891628,
 1.95735613493988,
 1.051856491790693,
 1.9252789093468063,
 1.0291944630801284,
 1.8730572354195854,
 1.7651409636517315,
 1.3336892935225801,
 1.3556333398076035,
 1.698649910567464,
 1.36

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.521505846263918,
 1.3030937891884438,
 1.8804267817666718,
 1.2952932506155252,
 1.07347433593599,
 1.9581950405845094,
 1.0036445215837255,
 1.0534476744021424,
 1.3927328810335962,
 1.8201831665923733,
 1.996397709360961,
 1.5311559752980601,
 1.4024572492564173,
 1.8624077658964548,
 1.4079272513466554,
 1.0224831983987843,
 1.265666142677329,
 1.5661694325855922,
 1.931193525777239,
 1.0052732002671783,
 1.983080338551026,
 1.1826952898547924,
 1.1773018712991627,
 1.9495430074916478,
 1.8322054821307434,
 1.3060572015506011,
 1.1546928157237797,
 1.3003000785471963,
 1.6942119542776615,
 1.6564715920156585,
 1.874229450019334,
 1.1330212177981271,
 1.8899019982112417,
 1.5197926790816947,
 1.2768513997338227,
 1.431803948484622,
 1.1238340307711894,
 1.0901185553982038,
 1.219190538552295,
 1.2443799052239737,
 1.3570120624387016,
 1.2737568494651827,
 1.3332655026641385,
 1.625370815986166,
 1.108236743030798,
 1.0939616573470623,
 1.88018503184807,
 1.1314380239437414,
 1.031

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 316 ms, sys: 51.3 ms, total: 368 ms
Wall time: 2.24 s
